# Amazon Bedrock AgentCore Observability：AgentCore Runtime 外でホストされた LlamaIndex エージェント

このノートブックでは、Amazon Bedrock AgentCore Runtime の外部でホストされた [LlamaIndex](https://www.llamaindex.ai/) エージェントの可観測性を設定する方法を説明します。セットアップが完了すると、Amazon CloudWatch の GenAI Observability ダッシュボードで LlamaIndex エージェントの内部意思決定プロセスを確認できるようになります。

## 学習内容
- Amazon OpenTelemetry Python インストルメンテーションで LlamaIndex エージェントを設定する方法
- CloudWatch GenAI Observability でエージェントトレースを可視化および分析する方法


## 前提条件
- Amazon CloudWatch で Transaction Search を有効にしてください。初回ユーザーは Bedrock AgentCore のスパンとトレースを表示するために CloudWatch Transaction Search を有効にする必要があります。Transaction Search を有効にするには、[ドキュメント](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html)を参照してください。
- Amazon CloudWatch でロググループとログストリームを設定し、環境変数に追加してください。
- Claude Haiku 4.5（モデル ID: global.anthropic.claude-haiku-4-5-20251001-v1:0）への Amazon Bedrock モデルアクセスを持つ AWS アカウント
- `aws configure` で AWS 認証情報を設定済み
- `LlamaIndex/.env.example` を参考に環境変数を設定した .env ファイル

## 1. セットアップとインストール

まず、必要な依存関係をインストールしましょう。requirements.txt ファイルに `aws-opentelemetry-distro` が含まれていることを確認してください。

#### 前提条件のデプロイ

開始する前に、AgentCore 可観測性用のロググループとログストリームを作成しましょう。

## 2. 環境設定
LlamaIndex エージェントの可観測性を有効にし、テレメトリデータを Amazon CloudWatch に送信するには、以下の環境変数を設定する必要があります。`.env` ファイルを使用してこれらの設定を安全に管理し、機密性の高い AWS 認証情報をコードから分離しながら、異なる環境間で簡単に切り替えられるようにします。

AWS 認証情報と設定を含む `.env` ファイルを作成してください。テンプレートとして `LlamaIndex/.env.example` を使用してください。

既存の `log group` と対応する `log stream` を使用する場合は、それを環境変数に追加してください。

そうでない場合は、環境変数として設定する前に、CloudWatch でロググループとログストリームを**作成**する必要があります。サンプル名を提供しています。

必要な環境変数:

| 変数 | 値 | 目的 |
|----------|-------|---------|
| `OTEL_PYTHON_DISTRO` | `aws_distro` | AWS Distro for OpenTelemetry (ADOT) を使用 |
| `OTEL_PYTHON_CONFIGURATOR` | `aws_configurator` | ADOT SDK 用の AWS コンフィギュレーターを設定 |
| `OTEL_EXPORTER_OTLP_PROTOCOL` | `http/protobuf` | エクスポートプロトコルを設定 |
| `OTEL_EXPORTER_OTLP_LOGS_HEADERS` | `x-aws-log-group=<YOUR-LOG-GROUP>,x-aws-log-stream=<YOUR-LOG-STREAM>,x-aws-metric-namespace=<YOUR-NAMESPACE>` | ログを CloudWatch グループに送信 |
| `OTEL_RESOURCE_ATTRIBUTES` | `service.name=<YOUR-AGENT-NAME>` | 可観測性データでエージェントを識別 |
| `AGENT_OBSERVABILITY_ENABLED` | `true` | ADOT パイプラインを有効化 |
| `AWS_REGION` | `<YOUR-REGION>` | AWS リージョン |

## 3. 環境変数の読み込み

`.env` ファイルから環境変数を読み込みましょう:

#### Transaction Search の有効化

この例を実行するには、まず Transaction Search を有効にする必要があります。AWS コンソールでこの[リンク](https://console.aws.amazon.com/cloudwatch/home#xray:settings/transaction-search)から有効化できます。

このページで、編集をクリックし、OpenTelemetry 形式で構造化ログとしてスパンを取り込むオプションを設定します。

![image.png](./images/transactional_search.png)
![image.png](./images/transactional_search2.png)

## 4. Python ファイルで LlamaIndex エージェントを作成

LlamaIndex Travel Agent の実装は `llamaindex_travel_agent.py` に提供されています。Amazon Bedrock のモデルでセットアップされた Travel Agent です。LlamaIndex エージェントは OpenTelemetry でセットアップされ、`opentelemetry-instrument` コマンドを使用する際に AWS OpenTelemetry distro がトレーサープロバイダーのセットアップを自動的に処理します。

このエージェントはシンプルな旅行推奨エージェントで、以下を行います:

- AWS Bedrock の Claude Haiku モデルを使用して会話型 AI エージェントを作成
- 会話フローに LlamaIndex の AgentWorkflow を使用
- DuckDuckGo を使用した Web 検索機能を統合
- エージェントの決定に基づいてツール呼び出しとルーティングを処理
- ユーザークエリに基づいて旅行推奨を返す

エージェントは以下で設定されています:

- 会話管理用の LlamaIndex Agent ワークフロー
- 推論とレスポンス生成のための FunctionCallingAgent
- Amazon Bedrock の Claude Haiku モデルを Large Language Model として使用
- リアルタイム情報取得のための Web 検索ツール
- セッションベースの会話トラッキング

`workflow` は、ファイル内で `workflow.run` を使用して作成・実行されます。

In [ ]:
%%writefile .env
# AWS OpenTelemetry Distribution
OTEL_PYTHON_DISTRO=aws_distro
OTEL_PYTHON_CONFIGURATOR=aws_configurator

# Export Protocol
OTEL_EXPORTER_OTLP_PROTOCOL=http/protobuf
OTEL_TRACES_EXPORTER=otlp

# CloudWatch Integration (uncomment and configure as needed)
OTEL_EXPORTER_OTLP_LOGS_HEADERS=x-aws-log-group=agents/llama-index-agent-logs10,x-aws-log-stream=default,x-aws-metric-namespace=bedrock-agentcore

# Service Identification
OTEL_RESOURCE_ATTRIBUTES=service.name=agentic-llamaindex-agentcore
# Enable Agent Observability
AGENT_OBSERVABILITY_ENABLED=true

# Disable instrumentations to get rid of span noise (OPTIONAL)
OTEL_PYTHON_DISABLED_INSTRUMENTATIONS=jinja2

## 5. AWS OpenTelemetry Python Distro

環境が設定されたので、可観測性がどのように機能するかを理解しましょう。[AWS OpenTelemetry Python Distro](https://pypi.org/project/aws-opentelemetry-distro/) は、コード変更なしでテレメトリデータをキャプチャするために LlamaIndex エージェントを自動的にインストルメントします。

このディストリビューションは以下を提供します:
- AgentCore Runtime 外でホストされた LlamaIndex エージェント（EC2、Lambda など）の**自動インストルメンテーション**
- シームレスな CloudWatch 統合のための **AWS 最適化設定**

### インストルメント済みエージェントの実行

LlamaIndex エージェントからトレースをキャプチャするには、Python を直接実行する代わりに `opentelemetry-instrument` コマンドを使用します。これにより、`.env` ファイルの環境変数を使用してインストルメンテーションが自動的に適用されます:

```bash
opentelemetry-instrument python llamaindex_travel_agent.py
```

このコマンドは:

- .env ファイルから OTEL 設定を読み込みます
- LlamaIndex、Amazon Bedrock 呼び出し、エージェントツールとデータベース、およびエージェントが行う他のリクエストを自動的にインストルメントします
- トレースを CloudWatch に送信します
- GenAI Observability ダッシュボードでエージェントの意思決定プロセスを可視化できるようにします

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Display the OTEL-related environment variables
otel_vars = [
    "OTEL_PYTHON_DISTRO",
    "OTEL_PYTHON_CONFIGURATOR",
    "OTEL_EXPORTER_OTLP_PROTOCOL",
    "OTEL_EXPORTER_OTLP_LOGS_HEADERS",
    "OTEL_RESOURCE_ATTRIBUTES",
    "AGENT_OBSERVABILITY_ENABLED",
    "OTEL_TRACES_EXPORTER",
    "OTEL_PYTHON_DISABLED_INSTRUMENTATIONS"
]

print("OpenTelemetry Configuration:")
for var in otel_vars:
    value = os.getenv(var)
    if value:
        print(f"{var}={value}")

## 6. セッショントラッキングの追加

複数のエージェント実行間でトレースを関連付けるには、OpenTelemetry baggage を使用してセッション ID をテレメトリデータに関連付けることができます:

```python
from opentelemetry import baggage, context
ctx = baggage.set_baggage("session.id", session_id)
```

セッション対応バージョンを実行:
```bash
opentelemetry-instrument python llamaindex_travel_agent_with_session.py --session-id "user-session-123"
```

## 7. 分析用カスタムメタデータ
フィルタリング、オフライン評価、およびパフォーマンス分析を可能にするためにカスタム属性を追加します。エージェントコードを変更して追加パラメータを受け入れる必要があります:

ctx = baggage.set_baggage("user.type", "premium")
ctx = baggage.set_baggage("experiment.id", "travel-agent-v2")
ctx = baggage.set_baggage("conversation.topic", "business-travel")
カスタムメタデータを使用したコマンド例:

```bash
# 異なる実験の A/B テスト
opentelemetry-instrument python agent.py --session-id "session-123" --experiment-id "model-a"
opentelemetry-instrument python agent.py --session-id "session-124" --experiment-id "model-b"

# 異なるユーザータイプのトラッキング
opentelemetry-instrument python agent.py --session-id "session-125" --user-type "premium"
opentelemetry-instrument python agent.py --session-id "session-126" --user-type "free"

# オフライン評価実行
opentelemetry-instrument python agent.py --session-id "eval-001" --dataset "golden-set-v1"
```

これらの属性は CloudWatch トレースに表示され、高度なフィルタリングと分析が可能になります。

In [ ]:
%%writefile llama_index_agent.py
###########################
#### エージェントコード: ####
###########################
import os
import asyncio
import logging
from llama_index.observability.otel import LlamaIndexOpenTelemetry
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent

# LlamaIndex 用の OpenTelemetry インストルメンテーションを初期化
instrumentor = LlamaIndexOpenTelemetry(debug=True)

# ロギングの設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# LlamaIndex ロギングの設定
logging.getLogger("llamaindex").setLevel(logging.INFO)

def multiply(a: int, b: int) -> int:
    """2 つの整数を掛け算し、結果の整数を返します"""
    return a * b


def add(a: int, b: int) -> int:
    """2 つの整数を足し算し、結果の整数を返します"""
    return a + b


def get_bedrock_model():
    model_id = os.getenv("BEDROCK_MODEL_ID", "global.anthropic.claude-haiku-4-5-20251001-v1:0")
    region = os.getenv("AWS_DEFAULT_REGION", "us-west-2")
    
    try:
        # boto3 に認証情報の解決を自動的に処理させる
        bedrock_model = BedrockConverse(
            model=model_id,
            region_name=region,
            # 明示的な認証情報なし - boto3 が自動的に見つけます
        )
        logger.info(f"Bedrock モデルを正常に初期化しました: {model_id}、リージョン: {region}")
        return bedrock_model
    except Exception as e:
        logger.error(f"Bedrock モデルの初期化に失敗しました: {str(e)}")
        logger.error("適切な AWS 認証情報が設定されていることと、Bedrock モデルへのアクセス権があることを確認してください")
        raise

# モデルを初期化
bedrock_model = get_bedrock_model()

# 算術エージェントを作成
agent = FunctionAgent(
    tools=[add, multiply],
    llm=bedrock_model,
)

# リスニングを開始
instrumentor.start_registering()

# 算術タスクを実行
query = """(121 + 2) * 5 は何ですか？"""

async def main():
    result = await agent.run(query)
    print("結果:", str(result))

asyncio.run(main())

## 5. AWS OpenTelemetry Python Distro

環境が設定されたので、可観測性がどのように機能するかを理解しましょう。[AWS OpenTelemetry Python Distro](https://pypi.org/project/aws-opentelemetry-distro/) は、コード変更なしでテレメトリデータをキャプチャするために LlamaIndex エージェントを自動的にインストルメントします。

このディストリビューションは以下を提供します:
- AgentCore Runtime 外でホストされた LlamaIndex エージェント（EC2、Lambda など）の**自動インストルメンテーション**
- シームレスな CloudWatch 統合のための **AWS 最適化設定**

### インストルメント済みエージェントの実行

LlamaIndex エージェントからトレースをキャプチャするには、Python を直接実行する代わりに `opentelemetry-instrument` コマンドを使用します。これにより、`.env` ファイルの環境変数を使用してインストルメンテーションが自動的に適用されます:

```bash
opentelemetry-instrument python llama_index_agent.py
```

このコマンドは:

- .env ファイルから OTEL 設定を読み込みます
- LlamaIndex、Amazon Bedrock 呼び出し、エージェントツールとデータベース、およびエージェントが行う他のリクエストを自動的にインストルメントします
- トレースを CloudWatch に送信します
- GenAI Observability ダッシュボードでエージェントの意思決定プロセスを可視化できるようにします

## 8. AWS CloudWatch で GenAI Observability ダッシュボードのトレースを理解する

LlamaIndex エージェントが OpenTelemetry インストルメンテーションで実行されると、AWS CloudWatch の GenAI Observability ダッシュボードでトレースを可視化および分析できます。Bedrock Agentcore に移動し、作成したエージェントをクリックします。

トレースには以下が表示されます:
- **ワークフロー実行**: LlamaIndex ワークフローのステップ
- **ツール呼び出し**: Web 検索操作とその結果
- **LLM インタラクション**: 入力/出力トークンを含む Bedrock モデル呼び出し
- **セッション相関**: セッション ID でグループ化された複数の実行

#### セッションページ:
![image.png](attachment:image.png)

#### トレースビューページ:
トレースビュー:
![image-4.png](attachment:image-4.png)

トレース詳細:
![image-2.png](attachment:image-2.png)
![image-3.png](attachment:image-3.png)

## 9. トラブルシューティング

Amazon CloudWatch でトレースが表示されない場合は、以下を確認してください:

1. **AWS 認証情報**: AWS 認証情報が正しく設定されていることを確認
2. **IAM 権限**: IAM ユーザー/ロールが CloudWatch の権限を持っていることを確認
3. **リージョンとロググループ**: 正しい AWS リージョンとロググループを確認していることを確認
4. **環境変数**: すべての OTEL_* 環境変数が正しく設定されていることを確認

## 10. まとめ

おめでとうございます！Amazon CloudWatch を通じて可観測性を持つ Amazon Bedrock モデルで LlamaIndex エージェントを実装してインストルメントしました。

- LlamaIndex トラベルエージェント
- AWS CloudWatch への完全な OpenTelemetry トレーシング
- Amazon Bedrock 呼び出し、LlamaIndex 操作などのトレース

## 11. 次のステップ

OpenTelemetry で LlamaIndex エージェントがセットアップされたので、以下を行うことができます:

1. **より複雑なワークフローを構築**: マルチエージェントワークフローを作成
2. **エージェントにツールを追加**: 様々な API ツールやカスタムツールを統合
3. **[アラームの設定](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/AlarmThatSendsEmail.html)**: `latency`、`token input`、`token output` などのビジネスにとって重要なメトリクスにアラームを作成

### AWS CloudWatch で GenAI Observability ダッシュボードのトレースを理解する（追加画像）

LlamaIndex エージェントが OpenTelemetry インストルメンテーションで実行されると、AWS CloudWatch の GenAI Observability ダッシュボードでトレースを可視化および分析できます。Bedrock Agentcore に移動し、作成したエージェントをクリックします。

#### セッションビューページ:

![llama_index_sessions.png](images/llama_index_sessions.png)


#### トレースビューページ:
トレースビュー:

![llama_index_sessions.png](images/llama_index_traces.png)


トレース詳細:

![llama_index_sessions.png](images/llama_index_trace_details.png)

### トラブルシューティング（追加情報）

Amazon CloudWatch または X-Ray でトレースが表示されない場合は、以下を確認してください:

1. **AWS 認証情報**: AWS 認証情報が正しく設定されていることを確認
2. **IAM 権限**: IAM ユーザー/ロールが CloudWatch の権限を持っていることを確認
3. **リージョン**: 正しい AWS リージョンを確認していることを確認
4. **環境変数**: すべての OTEL_* 環境変数が正しく設定されていることを確認

### まとめ（追加情報）

おめでとうございます！Amazon CloudWatch を通じて可観測性を持つ Amazon Bedrock モデルで LlamaIndex エージェントを実装してインストルメントしました。

- LlamaIndex 算術エージェント
- 完全な OpenTelemetry トレーシング
- Amazon Bedrock 呼び出し、LlamaIndex 操作などのトレース
- サービス名: agentic-llamaindex-agentcore

### 次のステップ（追加情報）

OpenTelemetry で LlamaIndex エージェントがセットアップされたので、以下を行うことができます:

1. **エージェントの追加**: 異なるパターンでマルチエージェントアーキテクチャを作成
2. **エージェントにツールを追加**: 検索ツール、API ツール、またはカスタムツールを統合
3. **[アラームの設定](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/AlarmThatSendsEmail.html)**: `latency`、`token input`、`token output` などのビジネスにとって重要なメトリクスにアラームを作成